# <노드 학습>
- 미니 프로젝트 주제: Movielens 1M Dataset을 기반으로, Session based Recommendation 시스템을 제작해본다.

In [1]:
import pandas
import tensorflow

print(pandas.__version__)
print(tensorflow.__version__)

1.3.3
2.6.0


In [2]:
import datetime as dt
from pathlib import Path
import os
import time
from datetime import datetime
from IPython.display import display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    # time 형식을 기존과 맞게 고쳐보기
    data['Time'] = pd.to_datetime(data['Time'], unit='s')
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


- 위에서 이전 학습(9-3(1))과 가장 크게 다른 부분 -> SessionID 대신 UserID 항목이 들어갔다는 점!
- 이 데이터셋은 명확한 1회 세션의 SessionID를 포함하지 않는다. 그래서 이번에는 UserID가 SessionID의 역할을 해야한다.
- 또한 Rating 정보가 포함되어 있음 -> Rating 정보가 포함됨으로써, 직전에 봤던 영화가 마음에 들었는지가 비슷한 영화를 더 고르게 하는 것과 상관이 있을 수도 있다! 
    - Rating이 낮은 데이터 또한 어떻게 처리할지도 고민해봐야 하는 문제이다.
- Time 항목에는 UTC time이 포함되어, 1970년 1월 1일부터 경과된 초 단위의 시간이 기재되어 있다.
- 이 내용들을 바탕으로, 한번 프로젝트를 해보자..!

## Step 1. 데이터 전처리
- cleanse_recursive(data, shortest, least_click) -> 짧은 세션과 인기가 없는 아이템을 반복적으로 제거한다.
    - while True 루프를 통해서, 데이터의 변화가 없을 때까지 전처리를 반복한다.
- cleanse_short_session(data: pd.DataFrame, shortest) -> UserId를 세션 단위로 보고, 각 사용자 세션의 길이를 계산하고 shortest 이상인 세션만 필터링한다.
- cleanse_unpopular_item(data: pd.DataFrame, least_click) -> 각 아이템이 얼마나 자주 클릭(평점)됐는지를 기준으로, least_click 미만인 아이템은 제거한다.

In [4]:
# 이부분을 이 데이터에 맞게 고치기

# short_session을 제거한 다음 unpopular item을 제거하면 다시 길이가 1인 session이 생길 수 있습니다.
# 이를 위해 반복문을 통해 지속적으로 제거 합니다.
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('UserId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['UserId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

In [5]:
# data = cleanse_recursive(data, shortest=2, least_click=5)
# data

- 그 다음, 세션 수(UserId 기준 고유값 수)를 출력한다.
- 여기서 nunique()를 활용하여, 전체 세션의 개수를 확인한다. (UserId를 SessionId처럼 사용함)
- 전체 중에서 세션 길이가 1인 세션의 비율을 확인한다.

In [6]:
# 1) 세션 길이 분포 확인
print("검증 세션 수:", data['UserId'].nunique())
print("세션별 길이(상위 10개):")
print(data.groupby('UserId').size().sort_values(ascending=False).head(10))

# 2) 길이 1인 세션 개수
print("길이 1 세션 비율:",
      (data.groupby('UserId').size() == 1).mean())

검증 세션 수: 6040
세션별 길이(상위 10개):
UserId
4169    2314
1680    1850
4277    1743
1941    1595
1181    1521
889     1518
3618    1344
2063    1323
1150    1302
1015    1286
dtype: int64
길이 1 세션 비율: 0.0


- 그 다음, split_by_date(data: pd.DataFrame, n_days: int) -> n_day를 기준으로 최근 데이터를 test 데이터 셋, 나머지를 train 데이터 셋으로 나눈다.
- Step1에서 전처리된 전체 데이터를 기준으로, 마지막 14일치 데이터를 test 데이터 셋으로 분리하고, 그 이전 데이터에서 다시 마지막 14일치를 validation 데이터 셋으로 분리하고, 나머지 데이터를 최종 train 데이터 셋 용도로 사용한다.

In [7]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    cutoff_time = final_time - dt.timedelta(days=n_days)

    # Train: cutoff 이전의 데이터
    train = data[data['Time'] < cutoff_time]

    # Test: cutoff 이후의 데이터, 단 아이템은 train에 있던 것만
    test = data[(data['Time'] >= cutoff_time) & (data['ItemId'].isin(train['ItemId']))]

    return train, test

In [8]:
tr, test = split_by_date(data, n_days=14)  # 마지막 이틀만 테스트용으로 분리
tr, val = split_by_date(tr, n_days=14)  # 마지막 이틀만 테스트용으로 분리

- 데이터 셋의 기초 통계 정보를 확인해본다.

In [9]:
# data에 대한 정보를 살펴봅니다.
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

# 클렌징을 추가로 진행해줘야함
- 이제, train/validation/test 세트에 대해서도 전처리를 적용하고, 이것들에 대해서도 기초 통계 정보를 확인해본다.
- 아래 코드는, 세션의 길이가 2보다 작은것을 제거하고, 적게 클릭된 아이템(train: 5회 미만 // val/test: 1회 미만)을 제거한다.
- 아래 코드에서 val, test는 너무 많은 데이터를 제거하지 않도록 하기 위해서 least_click=1로 설정한다.

In [10]:
tr = cleanse_recursive(tr, shortest=2, least_click=5)
val = cleanse_recursive(val, shortest=2, least_click=1)
test = cleanse_recursive(test, shortest=2, least_click=1)

In [11]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 998112
	 Sessions: 6040
	 Items: 3416
	 First Time : 2000-04-25 23:05:32
	 Last Time : 2003-01-31 12:16:30

* valid Set Stats Info
	 Events: 883
	 Sessions: 39
	 Items: 692
	 First Time : 2003-01-31 17:23:06
	 Last Time : 2003-02-14 02:02:32

* test Set Stats Info
	 Events: 575
	 Sessions: 44
	 Items: 489
	 First Time : 2003-02-15 00:15:09
	 Last Time : 2003-02-28 17:49:50



## Step 2. 미니 배치의 구성
- 먼저, 훈련 데이터(tr)에 등장하는 고유한 아이템Id를 기준으로, 각 아이템에 고유한 정수 인덱스를 부여한다.
- 그리고, 데이터프레임(tr, val, test)에 대해서 아이템Id를 정수 인덱스(item_idx)로 변환한다. 이때, id2idx에 없는 아이템은 -1로 처리하여 모델에 노출되지 않도록 한다. (콜드 스타트 문제 해결하기 위함)

In [12]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

- 그 다음, 검증 데이터 셋(val)에 대한 세션 분석을 수행한다.

In [13]:
# 1) 세션 길이 분포 확인
print("검증 세션 수:", val['UserId'].nunique())
print("세션별 길이(상위 10개):")
print(val.groupby('UserId').size().sort_values(ascending=False).head(10))

# 2) 길이 1인 세션 개수
print("길이 1 세션 비율:",
      (val.groupby('UserId').size() == 1).mean())

검증 세션 수: 39
세션별 길이(상위 10개):
UserId
398     220
3012    165
4958    133
419      79
5654     72
5172     61
195      13
4312     12
2648     12
3732     11
dtype: int64
길이 1 세션 비율: 0.0


In [14]:
tr

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,2000-12-31 22:00:19,0
22,1,1270,5,2000-12-31 22:00:55,1
27,1,1721,4,2000-12-31 22:00:55,2
37,1,1022,5,2000-12-31 22:00:55,3
24,1,2340,3,2000-12-31 22:01:43,4
...,...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29,1248
999988,6040,1921,4,2001-08-10 14:41:04,370
1000172,6040,1784,3,2001-08-10 14:41:04,89
1000167,6040,161,3,2001-08-10 14:41:26,464


- SessionDataset -> 세션 기반 추천 모델 학습을 위한 세션 단위 데이터셋 구성 클래스를 생성한다. 이는 세션별 클릭 순서를 빠르게 추출하기 위해 세션 인덱스, 오프셋 정보를 저장한다.
- 즉, 미니배치 구성을 위한 세션 인덱싱 구조를 준비하는 코드이다.

In [15]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

- 세션 구조가 잘 유지됐는지 확인하기 위해, 전처리된 데이터프레임의 상위 10개 행을 출력해본다. 

In [16]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,2000-12-31 22:00:19,0
22,1,1270,5,2000-12-31 22:00:55,1
27,1,1721,4,2000-12-31 22:00:55,2
37,1,1022,5,2000-12-31 22:00:55,3
24,1,2340,3,2000-12-31 22:01:43,4
36,1,1836,5,2000-12-31 22:02:52,5
3,1,3408,4,2000-12-31 22:04:35,6
7,1,2804,5,2000-12-31 22:11:59,7
47,1,1207,4,2000-12-31 22:11:59,8
0,1,1193,5,2000-12-31 22:12:40,9


- 아래 코드는, 각 UserId(세션)이 시작하는 인덱스 위치를 저장한 배열을 나타낸다.

In [17]:
tr_dataset.click_offsets

array([     0,     53,    182, ..., 997648, 997771, 998112], dtype=int32)

- 전체 세션의 인덱스를 0부터 차례대로 나열한 세션 번호 배열을 출력해본다.

In [18]:
tr_dataset.session_idx

array([   0,    1,    2, ..., 6037, 6038, 6039])

- 선색한 세션(여기서는 0,1,2,3)의 시작 위치와 종료 위치 바로 다음의 인덱스를 가져온다. 이를 통해 각 세션의 데이터를 start[i]:end[i] 범위로 슬라이싱할 수 있다.
    - start: 각 세션의 시작 인덱스 배열
    - end: 각 세션의 종료 인덱스 +1 위치 배열

In [19]:
start = tr_dataset.click_offsets[tr_dataset.session_idx[[[0,1,2,3]]]]       # data 상에서 session이 시작된 위치를 가져옵니다.
end = tr_dataset.click_offsets[tr_dataset.session_idx[[0,1,2,3]] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.

start
end

array([ 53, 182, 233, 254], dtype=int32)

- 아래 코드는, 선택한 세션 중 가장 짧은 세션의 길이에서 1을 뺀 값을 구함으로써, 모든 세션에서 입력과 라벨로 나눌 수 있는 최소 시퀀스 길이를 구한다.

In [21]:
(end - start).min() -1

20

- 각 세션 시작 위치(start)에 20을 더한 인덱스에서 item_idx 값을 추출한다.
- 즉, 각 세션의 21번째 아이템 인덱스들을 배열로 가져오는 코드이다.

In [22]:
inp = tr_dataset.df['item_idx'].values[start + 20]
inp

array([ 20,  71, 185,  88])

- 그 다음, 각 세션에서 21번째 입력 다음에 등장하는 아이템 인덱스들 즉, 예측 대상(target)을 추출한다.
- 이 코드는 start+20 위치의 inp를 보고, target인 start+21을 예측한다.

In [23]:
target = tr_dataset.df['item_idx'].values[start + 20 + 1]
target

array([ 21,  72, 186,  56])

- 아래 코드는 SessionDataLoader 클래스를 구현하여, 세션 데이터를 병렬로 처리할 수 있도록 입력과 타겟을 순차적으로 생성하며, 남은 세션의 수가 batch_size보다 적을 경우 자동으로 조정하여 처리 누락을 방지하는 코드이다.
- self.batch_size = min(batch_size, len(dataset.session_idx)) -> 남은 세션 수에 맞춰 배치 크기 자동 조정한다.
- __iter__ -> 세션별 item_idx를 이용해 입력/타깃 시퀀스를 생성한다.
- initialize / update_status -> 세션이 끝날 때마다 다음 세션으로 교체한다.

In [24]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""


    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        
## 이부분에 batch_size 를 마지막에 남은 부분으 처리할수있게 코드 작성
#         self.batch_size = batch_size
        self.batch_size = min(batch_size, len(dataset.session_idx))  # 이 줄 추가

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.

        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

- 미니배치 구성이 잘 될지 확인하기 위해, 세션 데이터 상위 15개를 출력하여 확인해본다.

In [25]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,2000-12-31 22:00:19,0
22,1,1270,5,2000-12-31 22:00:55,1
27,1,1721,4,2000-12-31 22:00:55,2
37,1,1022,5,2000-12-31 22:00:55,3
24,1,2340,3,2000-12-31 22:01:43,4
36,1,1836,5,2000-12-31 22:02:52,5
3,1,3408,4,2000-12-31 22:04:35,6
7,1,2804,5,2000-12-31 22:11:59,7
47,1,1207,4,2000-12-31 22:11:59,8
0,1,1193,5,2000-12-31 22:12:40,9


- SessionDataLoader에서 첫번째 미니 배치를 꺼내서, 모델 입력(inputs), 정답 라벨(labels), 종료된 세션 마스크(mask)를 출력한다.

In [26]:
iter_ex = iter(tr_data_loader)

inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0 53 65 54]
Label Item Idx are :       [ 1 54 62 24]
Previous Masked Input Idx are []


- 이제, 추천 결과 평가 지표인 MRR@k와 Recall@k를 계산한다.
    - mrr_k(pred, truth, k) -> 정답 아이템(truth)이 상위 k개 예측 중 몇 번째에 있는지를 보고, **역순위 평균(MRR)**을 반환한다.
    - recall_k(pred, truth, k) -> 정답 아이템이 상위 k개 예측 안에 있으면 1, 없으면 0을 반환한다.

In [27]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

## Step 3. 모델 구성

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

- 아래 코드의 create_model 함수 간단한 설명
    - 입력 -> (배치 크기, 타임스텝=1, 아이템 수) 형태의 원-핫 인코딩된 시퀀스 형태이다.
    - GRU 층에서 이 시퀀스를 처리하고, stateful=True로 설정하여, 세션간의 순서를 유지하면서 학습이 가능하게끔 한다.
    - 또한, 과적합 방지를 위해 드롭아웃을 넣었다.
    - 출력층 -> 모든 아이템에 대한 softmax 확률 분포를 출력한다.
    - 모델을 정의하고, CrossEntropy + Adam optimizer로 compile한다.

In [29]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

- 아래 코드는, 모델 학습 및 평가에 필요한 하이퍼파라미터와 데이터 셋을 한번에 담는 클래스를 정의한것이다.

In [30]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

## 배치사이즈를 너무 크게두면 밑에 평가 부분에서 nan이 나옵니다    
args = Args(tr, val, test, batch_size=256, hsz=50, drop_rate=0.1, lr=0.001, epochs=3, k=20)

In [31]:
model = create_model(args)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(256, 1, 3416)]          0         
_________________________________________________________________
GRU (GRU)                    [(256, 50), (256, 50)]    520200    
_________________________________________________________________
dropout (Dropout)            (256, 50)                 0         
_________________________________________________________________
dense (Dense)                (256, 3416)               174216    
Total params: 694,416
Trainable params: 694,416
Non-trainable params: 0
_________________________________________________________________


- 여기까지 했으면, 일단 모델이 학습할 준비는 다 마친 상태이다..!

## Step 4. 모델 학습
- 아래 코드는 GRU 모델을 세션 기반 미니배치로 학습하고, 각 epoch마다 검증 데이터(val)로 Recall@k, MRR@k를 계산하여 성능을 평가하는 코드이다. 여기서 GRU의 stateful=True 특성상, 세션 종료마다 hidden state를 초기화한다.
- train_model(model, args) -> 학습 루프를 돌며, train 데이터로 학습하고, validation 데이터로 성능을 측정한다. 각 배치마다 세션이 종료된 위치의 hidden state를 초기화하여 GRU의 상태를 정확하게 유지하도록 한다.
- reset_hidden_states(model, mask) -> mask에 해당하는 세션의 GRU hidden state만 선별적으로 0으로 초기화한다.
- get_metrics(data, model, args, k) -> validation/test 데이터로 예측하고, Recall@k, MRR@k를 계산한다. 이때, 마지막 배치 크기가 부족할 경우 0으로 패딩하여 모델에 입력하도록 한다.

In [32]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    tf.config.run_functions_eagerly(True)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['UserId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다.
                                                     # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['UserId'].nunique()

    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1) 
        # 
        if input_ohe.shape[0] < args.batch_size:
            pad_len = args.batch_size - input_ohe.shape[0]
            padding = np.zeros((pad_len, 1, args.num_items))
            input_ohe = np.concatenate([input_ohe, padding], axis=0)

        pred = model.predict(input_ohe, batch_size=args.batch_size)


        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])
    
    
    print(recall_list)
    print(mrr_list)
    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [33]:
train_model(model, args)

Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.64it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 1: 0.025641
	 - MRR@20    epoch 1: 0.008547



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.96it/s]


[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.05263157894736842, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 2: 0.076923
	 - MRR@20    epoch 2: 0.026991



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  5.00it/s]

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0.08333333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.07142857142857142, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.14285714285714285, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 3: 0.076923
	 - MRR@20    epoch 3: 0.007631



- 여기까지가 위에서 정의한 GRU 모델을 설정한 파라미터에 따라 학습을 진행한것이다. 
- 위의 결과들은, 훈련 데이터로 train_on_batch() 학습을 진행하고, 매 epoch마다 검증 성능을 출력한것이다.

## Step 5. 모델 테스트
- 아래 코드는 학습이 완료된 모델을 테스트 데이터로 평가하여, 최종 성능 지표인 Recall@20과 MRR@20을 출력하는 코드이다.

In [34]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)

Evaluation:  50%|█████     | 1/2 [00:00<00:00,  4.47it/s]

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0.14285714285714285, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.125, 0, 0, 0.07692307692307693, 0, 0, 0]
	 - Recall@20: 0.068182
	 - MRR@20: 0.007836



------------------------------------------------------------------------------

# [실험 4가지]
- 실험 1) GRU hidden size 증가시키기 (hsz: 50(기존) -> 100)
    - GRU의 hidden state 크기를 늘리면, 더 많은 정보를 보존할 수 있음 -> 더욱더 정교한 사용자의 행동을 예측 가능하다.
- 실험 2) Learning Rate 증가시키기 (lr: 0.001(기존) -> 0.005)
    - 학습 속도를 빠르게 하여 더 빠른 수렴을 만들도록 함
- 실험 3) Dropout 증가시키기 (drop_rate: 0.1(기존) -> 0.3)
- 실험 4) 패딩 방식 개선시키기
    - 세션이 끝날 때 마지막 항목 기준으로 padding을 하여 GRU state를 더 적절하게 유지하도록 개선함
    - 전처리 시 순서/패딩 전략을 개선한 tr, val, test 사용함
    - 나머지 파라미터는 위의 baseline과 동일함

## 실험 1) GRU hidden size 증가시키기 (hsz: 50(기존) -> 100)

In [37]:
print("======== 실험 1: GRU hidden size 증가 ========")

args_exp1 = Args(
    tr=tr,
    val=val,
    test=test,
    batch_size=256,
    hsz=100,  # 변경됨
    drop_rate=0.1,
    lr=0.001,
    epochs=3,
    k=20
)

model_exp1 = create_model(args_exp1)
train_model(model_exp1, args_exp1)
recall_exp1, mrr_exp1 = get_metrics(test, model_exp1, args_exp1, args_exp1.k)

print(f"[실험 1 결과] Recall@20: {recall_exp1:.4f}, MRR@20: {mrr_exp1:.4f}")

======== 실험 1: GRU hidden size 증가 ========
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(256, 1, 3416)]          0         
_________________________________________________________________
GRU (GRU)                    [(256, 100), (256, 100)]  1055400   
_________________________________________________________________
dropout_5 (Dropout)          (256, 100)                0         
_________________________________________________________________
dense_5 (Dense)              (256, 3416)               345016    
Total params: 1,400,416
Trainable params: 1,400,416
Non-trainable params: 0
_________________________________________________________________


Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.87it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1111111111111111, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 1: 0.051282
	 - MRR@20    epoch 1: 0.015670



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.92it/s]


[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0.07692307692307693, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.14285714285714285, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 2: 0.076923
	 - MRR@20    epoch 2: 0.010764



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.85it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1111111111111111, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 3: 0.051282
	 - MRR@20    epoch 3: 0.007123



Evaluation:  50%|█████     | 1/2 [00:00<00:00,  4.51it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.125, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.125, 0, 0, 0, 0, 0.06666666666666667, 0]
[실험 1 결과] Recall@20: 0.0682, MRR@20: 0.0072


## 실험 2) Learning Rate 증가시키기 (lr: 0.001(기존) -> 0.005)

In [38]:
print("======== 실험 2: Learning Rate 증가 ========")

args_exp2 = Args(
    tr=tr,
    val=val,
    test=test,
    batch_size=256,
    hsz=50,
    drop_rate=0.1,
    lr=0.005,  # 변경됨
    epochs=3,
    k=20
)

model_exp2 = create_model(args_exp2)
train_model(model_exp2, args_exp2)
recall_exp2, mrr_exp2 = get_metrics(test, model_exp2, args_exp2, args_exp2.k)

print(f"[실험 2 결과] Recall@20: {recall_exp2:.4f}, MRR@20: {mrr_exp2:.4f}")

======== 실험 2: Learning Rate 증가 ========
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(256, 1, 3416)]          0         
_________________________________________________________________
GRU (GRU)                    [(256, 50), (256, 50)]    520200    
_________________________________________________________________
dropout_6 (Dropout)          (256, 50)                 0         
_________________________________________________________________
dense_6 (Dense)              (256, 3416)               174216    
Total params: 694,416
Trainable params: 694,416
Non-trainable params: 0
_________________________________________________________________


Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.84it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 1: 0.051282
	 - MRR@20    epoch 1: 0.006838



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.95it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 2: 0.025641
	 - MRR@20    epoch 2: 0.006410



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.96it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 3: 0.025641
	 - MRR@20    epoch 3: 0.005128



Evaluation:  50%|█████     | 1/2 [00:00<00:00,  4.59it/s]

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.14285714285714285, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.25, 0, 0, 0.05555555555555555, 0, 0, 0]
[실험 2 결과] Recall@20: 0.0909, MRR@20: 0.0140


## 실험 3) Dropout 증가시키기 (drop_rate: 0.1(기존) -> 0.3)

In [39]:
print("======== 실험 3: Dropout 증가 ========")

args_exp3 = Args(
    tr=tr,
    val=val,
    test=test,
    batch_size=256,
    hsz=50,
    drop_rate=0.3,  # 변경됨
    lr=0.001,
    epochs=3,
    k=20
)

model_exp3 = create_model(args_exp3)
train_model(model_exp3, args_exp3)
recall_exp3, mrr_exp3 = get_metrics(test, model_exp3, args_exp3, args_exp3.k)

print(f"[실험 3 결과] Recall@20: {recall_exp3:.4f}, MRR@20: {mrr_exp3:.4f}")

======== 실험 3: Dropout 증가 ========
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(256, 1, 3416)]          0         
_________________________________________________________________
GRU (GRU)                    [(256, 50), (256, 50)]    520200    
_________________________________________________________________
dropout_7 (Dropout)          (256, 50)                 0         
_________________________________________________________________
dense_7 (Dense)              (256, 3416)               174216    
Total params: 694,416
Trainable params: 694,416
Non-trainable params: 0
_________________________________________________________________


Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.91it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 1: 0.025641
	 - MRR@20    epoch 1: 0.006410



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  5.04it/s]


[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0.06666666666666667, 0, 0]
	 - Recall@20 epoch 2: 0.076923
	 - MRR@20    epoch 2: 0.027350



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  5.02it/s]


[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.06666666666666667, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 3: 0.076923
	 - MRR@20    epoch 3: 0.040171



Evaluation:  50%|█████     | 1/2 [00:00<00:00,  4.47it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0]
[실험 3 결과] Recall@20: 0.0227, MRR@20: 0.0038


## 실험 4) 패딩 방식 개선시키기

In [40]:
print("======== 실험 4: 패딩 및 순서 보정 적용 ========")

args_exp4 = Args(
    tr=tr,  # padding/순서 개선된 데이터셋
    val=val,
    test=test,
    batch_size=256,
    hsz=50,
    drop_rate=0.1,
    lr=0.001,
    epochs=3,
    k=20
)

model_exp4 = create_model(args_exp4)
train_model(model_exp4, args_exp4)
recall_exp4, mrr_exp4 = get_metrics(test, model_exp4, args_exp4, args_exp4.k)

print(f"[실험 4 결과] Recall@20: {recall_exp4:.4f}, MRR@20: {mrr_exp4:.4f}")

======== 실험 4: 패딩 및 순서 보정 적용 ========
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(256, 1, 3416)]          0         
_________________________________________________________________
GRU (GRU)                    [(256, 50), (256, 50)]    520200    
_________________________________________________________________
dropout_8 (Dropout)          (256, 50)                 0         
_________________________________________________________________
dense_8 (Dense)              (256, 3416)               174216    
Total params: 694,416
Trainable params: 694,416
Non-trainable params: 0
_________________________________________________________________


Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  5.02it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.125, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 1: 0.051282
	 - MRR@20    epoch 1: 0.016026



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.85it/s]


[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.07692307692307693, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 2: 0.076923
	 - MRR@20    epoch 2: 0.014793



Evaluation:  33%|███▎      | 1/3 [00:00<00:00,  4.87it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.09090909090909091, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 - Recall@20 epoch 3: 0.051282
	 - MRR@20    epoch 3: 0.006605



Evaluation:  50%|█████     | 1/2 [00:00<00:00,  4.54it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.058823529411764705, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0, 0, 0.05263157894736842, 0, 0, 0]
[실험 4 결과] Recall@20: 0.0682, MRR@20: 0.0071


----------------------------------------------------------------------------------------------------------

### [실험 결과 및 정리]
- 기존 Baseline
    - Recall@20 = 0.0682, MRR@20 = 0.0078
    - MRR은 낮지만 추천 시스템 기준으로는 나름 합리적인 Baseline 이라고 판단할 수 있다..!
- 실험1) GRU hidden size 증가
    - 변화 없음 (Recall 동일, MRR 약간 감소함)
    - 모델 복잡도 증가가 성능에 크게 영향을 주지 않았음 → 오히려 미세하게 과적합 또는 gradient 소실 가능성을 보였다.
- 실험 2) Learning Rate 증가
    - Recall 0.0909 / MRR 0.0140 → 크게 성능 향상을 보였다!
    - 가장 효과적인 실험으로 판단할 수 있다.
    - 이는 학습 속도가 적절하게 증가해 빠르게 수렴하고 일반화 성능이 향상됐다고 볼 수 있다.
- 실험 3) Dropout 증가
    - Recall 0.0227 / MRR 0.0038 → 성능이 급감하였다.
    - 과도한 정규화(?) → 모델이 충분히 학습하지 못한것을 알 수 있다.
    - Dropout도 적당히 조절할 필요가 있음을 알 수 있다.
- 실험 4) 패딩 방식 개선시키기
    - Recall 동일 / MRR 약간 감소 -> 단순한 순서의 조정이 GRU 기반 구조에서는 통하지 않았음을 알 수 있다.